In [68]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

In [69]:

annotations_path = r'C:\Users\Admin\Desktop\CapstoneProject\capstone\ExDark\ExDark\annotationsCSVfile.csv'
annotations = pd.read_csv(annotations_path)

dataset_path = r'C:\Users\Admin\Desktop\CapstoneProject\capstone\ExDark\AnotedImages'
annotations['image_path'] = annotations['image_path'].apply(lambda x: os.path.join(dataset_path, x))



In [70]:

image_paths = annotations['image_path'].tolist()
bounding_boxes = annotations[['x_tl', 'y_tl', 'x_br', 'y_br']].values.tolist()
class_labels = annotations['class'].tolist()

image_paths_train, image_paths_val, bounding_boxes_train, bounding_boxes_val, class_labels_train, class_labels_val = train_test_split(
    image_paths, bounding_boxes, class_labels, test_size=0.2, random_state=42
)


In [71]:
def preprocess_image(image_path, bbox, class_label):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array /= 255.0

    # Normalize bounding box coordinates to the range [0, 1]
    bbox = [
        bbox[0] / img_array.shape[1],
        bbox[1] / img_array.shape[0],
        bbox[2] / img_array.shape[1],
        bbox[3] / img_array.shape[0]
    ]

    return img_array, {'boxes': [bbox], 'labels': [class_label]}


In [72]:
def build_resnet_backbone(input_shape=(224, 224, 3)):
    base_model = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    backbone = keras.Model(inputs=base_model.input, outputs=base_model.get_layer('conv4_block6_out').output, name='resnet_backbone')
    return backbone


In [73]:
def build_rpn(input_tensor, num_anchors=9):
    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu', kernel_initializer='normal', name='rpn_conv')(input_tensor)
    rpn_cls = layers.Conv2D(num_anchors * 2, (1, 1), activation='linear', name='rpn_cls')(x)
    rpn_reg = layers.Conv2D(num_anchors * 4, (1, 1), activation='linear', name='rpn_reg')(x)

    model = keras.Model(inputs=input_tensor, outputs=[rpn_cls, rpn_reg], name='rpn_model')
    return model

In [81]:
def build_classifier(base_model, num_classes, input_shape):
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)

    # Calculate the total size of the flattened feature vector
    flattened_size = int(np.prod(input_shape[1:]))  # Exclude the batch size dimension
    
    x = layers.Dense(flattened_size, activation='softmax', name='cls_output')(x)

    # Reshape x to match the specified input_shape
    x = layers.Reshape((input_shape[1], input_shape[2], -1))(x)

    model = keras.Model(inputs=base_model.input, outputs=x, name='classifier_model')
    return model


In [82]:

def build_faster_rcnn(input_shape=(224, 224, 3), num_anchors=9):
    # ResNet Backbone
    resnet_backbone = build_resnet_backbone(input_shape)
    rpn_input = resnet_backbone.output

    # RPN Model
    rpn_model = build_rpn(rpn_input, num_anchors)
    rpn_cls_output, _ = rpn_model(rpn_input)

    # Classifier Input
    classifier_input = layers.Input(shape=(7, 7, num_anchors * 4), name='classifier_input')

    # Build classifier model with correct input shape
    classifier_model = build_classifier(base_model=resnet_backbone, num_classes=num_classes, input_shape=(7, 7, num_anchors * 4))

    # Pass rpn_cls_output directly to classifier_model
    roi_features = classifier_model(rpn_cls_output)

    # Faster RCNN Model
    faster_rcnn_model = keras.Model(
        inputs=resnet_backbone.input,
        outputs=[rpn_model(rpn_input), roi_features],
        name='faster_rcnn_model'
    )

    return faster_rcnn_model

In [83]:
num_classes = 11
num_anchors = 9

faster_rcnn = build_faster_rcnn()
faster_rcnn.compile(optimizer='adam', loss={'rpn_model': 'binary_crossentropy', 'classifier_model': 'categorical_crossentropy'})
faster_rcnn.summary()

ValueError: Exception encountered when calling layer "classifier_model" (type Functional).

Input 0 of layer "conv1_conv" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (None, 20, 20, 18)

Call arguments received by layer "classifier_model" (type Functional):
  • inputs=tf.Tensor(shape=(None, 14, 14, 18), dtype=float32)
  • training=None
  • mask=None

In [ ]:

# train_dataset = tf.data.Dataset.from_tensor_slices((image_paths_train, bounding_boxes_train, class_labels_train))
# train_dataset = train_dataset.map(preprocess_image)
